# Workflow Orchestration - Chaînage Multi-Modèles

**Module :** 03-Images-Orchestration  
**Niveau :** Expert  
**Durée estimée :** 60 minutes  

## Introduction

L'orchestration de workflows permet de combiner plusieurs modèles et opérations pour créer des pipelines de génération d'images sophistiqués. Ce notebook couvre:

- **Chaînage séquentiel**: Text → Image → Edit → Upscale
- **Exécution parallèle**: Génération simultanée multi-modèles
- **Workflows conditionnels**: Branchement selon qualité/contenu
- **Gestion des erreurs**: Retry, fallback, timeouts

### Architecture d'Orchestration

```
┌─────────────────────────────────────────────────────────┐
│               Workflow Orchestrator                      │
├─────────────────────────────────────────────────────────┤
│                                                          │
│   ┌─────────┐    ┌─────────┐    ┌─────────┐            │
│   │ Prompt  │───▶│ Model A │───▶│ Model B │───▶ Output │
│   └─────────┘    └─────────┘    └─────────┘            │
│                        │                                 │
│                        ▼                                 │
│                  ┌─────────┐                            │
│                  │ Model C │ (parallel)                 │
│                  └─────────┘                            │
│                                                          │
└─────────────────────────────────────────────────────────┘
```

## Prérequis

- Modules 01 et 02 complétés
- Accès aux services ComfyUI et/ou APIs cloud
- Compréhension des différents modèles (Qwen, FLUX, SD3.5)

In [ ]:
# =============================================================================
# 1. CONFIGURATION ET IMPORTS
# =============================================================================

import os
import sys
import time
import json
import asyncio
import hashlib
from io import BytesIO
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass, field
from typing import Optional, Dict, List, Tuple, Any, Callable, Union
from enum import Enum
from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import requests

# Chargement variables d'environnement
from dotenv import load_dotenv
load_dotenv("../.env")
load_dotenv("../00-GenAI-Environment/.env")

# Configuration
COMFYUI_URL = os.getenv("COMFYUI_API_URL", "http://localhost:8188")
COMFYUI_TOKEN = os.getenv("COMFYUI_AUTH_TOKEN")

print("╔════════════════════════════════════════════════════╗")
print("║   Workflow Orchestration - Multi-Model Pipelines  ║")
print("╚════════════════════════════════════════════════════╝")
print(f"\n📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# =============================================================================
# 2. TYPES ET STRUCTURES DE DONNÉES
# =============================================================================

class TaskStatus(Enum):
    PENDING = "pending"
    RUNNING = "running"
    COMPLETED = "completed"
    FAILED = "failed"
    SKIPPED = "skipped"

@dataclass
class TaskResult:
    """Résultat d'une tâche d'orchestration."""
    task_id: str
    status: TaskStatus
    output: Any = None
    error: str = None
    duration: float = 0.0
    metadata: Dict = field(default_factory=dict)

@dataclass
class WorkflowStep:
    """Étape d'un workflow."""
    name: str
    func: Callable
    inputs: Dict = field(default_factory=dict)
    depends_on: List[str] = field(default_factory=list)
    retry_count: int = 3
    timeout: float = 120.0
    condition: Callable = None  # Exécuter si condition() retourne True

print("✅ Types et structures définis")

In [ ]:
# =============================================================================
# 3. ORCHESTRATEUR DE WORKFLOWS
# =============================================================================

class WorkflowOrchestrator:
    """
    Orchestrateur pour pipelines de génération d'images.
    
    Fonctionnalités:
    - Exécution séquentielle et parallèle
    - Gestion des dépendances entre étapes
    - Retry automatique avec backoff
    - Timeouts configurables
    - Caching des résultats intermédiaires
    """
    
    def __init__(self, max_workers: int = 4):
        self.max_workers = max_workers
        self.results: Dict[str, TaskResult] = {}
        self.cache: Dict[str, Any] = {}
        self.execution_log: List[Dict] = []
    
    def _generate_cache_key(self, step_name: str, inputs: Dict) -> str:
        """Génère une clé de cache unique."""
        content = f"{step_name}:{json.dumps(inputs, sort_keys=True)}"
        return hashlib.md5(content.encode()).hexdigest()
    
    def _execute_with_retry(self, step: WorkflowStep, inputs: Dict) -> TaskResult:
        """Exécute une étape avec retry et timeout."""
        task_id = f"{step.name}_{int(time.time()*1000)}"
        
        # Vérifier le cache
        cache_key = self._generate_cache_key(step.name, inputs)
        if cache_key in self.cache:
            print(f"   📦 Cache hit pour {step.name}")
            return TaskResult(
                task_id=task_id,
                status=TaskStatus.COMPLETED,
                output=self.cache[cache_key],
                metadata={"cached": True}
            )
        
        # Vérifier la condition
        if step.condition and not step.condition(inputs):
            print(f"   ⏭️ Skipping {step.name} (condition non remplie)")
            return TaskResult(task_id=task_id, status=TaskStatus.SKIPPED)
        
        # Exécution avec retry
        last_error = None
        for attempt in range(step.retry_count):
            try:
                start = time.time()
                print(f"   🔄 {step.name} (attempt {attempt + 1}/{step.retry_count})")
                
                output = step.func(**inputs)
                duration = time.time() - start
                
                # Mise en cache
                self.cache[cache_key] = output
                
                return TaskResult(
                    task_id=task_id,
                    status=TaskStatus.COMPLETED,
                    output=output,
                    duration=duration
                )
                
            except Exception as e:
                last_error = str(e)
                print(f"   ⚠️ Erreur: {last_error}")
                if attempt < step.retry_count - 1:
                    wait_time = 2 ** attempt  # Exponential backoff
                    print(f"   ⏳ Retry dans {wait_time}s...")
                    time.sleep(wait_time)
        
        return TaskResult(
            task_id=task_id,
            status=TaskStatus.FAILED,
            error=last_error
        )
    
    def run_sequential(self, steps: List[WorkflowStep], initial_input: Dict = None) -> Dict[str, TaskResult]:
        """
        Exécute les étapes séquentiellement.
        Le résultat de chaque étape est passé à la suivante.
        """
        print("\n🔗 Exécution Séquentielle")
        print("=" * 40)
        
        self.results = {}
        current_input = initial_input or {}
        
        for i, step in enumerate(steps):
            print(f"\n[{i+1}/{len(steps)}] {step.name}")
            
            # Merge inputs
            merged_inputs = {**current_input, **step.inputs}
            
            result = self._execute_with_retry(step, merged_inputs)
            self.results[step.name] = result
            
            if result.status == TaskStatus.FAILED:
                print(f"   ❌ Pipeline arrêté sur {step.name}")
                break
            elif result.status == TaskStatus.COMPLETED:
                print(f"   ✅ Complété en {result.duration:.2f}s")
                # Passer le résultat à l'étape suivante
                if result.output is not None:
                    if isinstance(result.output, dict):
                        current_input.update(result.output)
                    else:
                        current_input["previous_output"] = result.output
        
        return self.results
    
    def run_parallel(self, steps: List[WorkflowStep], shared_input: Dict = None) -> Dict[str, TaskResult]:
        """
        Exécute les étapes en parallèle.
        Toutes les étapes reçoivent le même input.
        """
        print("\n⚡ Exécution Parallèle")
        print("=" * 40)
        
        self.results = {}
        shared_input = shared_input or {}
        
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            futures = {}
            
            for step in steps:
                merged_inputs = {**shared_input, **step.inputs}
                future = executor.submit(self._execute_with_retry, step, merged_inputs)
                futures[future] = step.name
            
            for future in as_completed(futures):
                step_name = futures[future]
                result = future.result()
                self.results[step_name] = result
                
                status_icon = "✅" if result.status == TaskStatus.COMPLETED else "❌"
                print(f"   {status_icon} {step_name}: {result.status.value}")
        
        return self.results
    
    def get_summary(self) -> Dict:
        """Retourne un résumé de l'exécution."""
        completed = sum(1 for r in self.results.values() if r.status == TaskStatus.COMPLETED)
        failed = sum(1 for r in self.results.values() if r.status == TaskStatus.FAILED)
        total_time = sum(r.duration for r in self.results.values())
        
        return {
            "total_steps": len(self.results),
            "completed": completed,
            "failed": failed,
            "total_time": total_time,
            "cache_hits": sum(1 for r in self.results.values() if r.metadata.get("cached"))
        }

# Instanciation
orchestrator = WorkflowOrchestrator(max_workers=4)
print("\n✅ WorkflowOrchestrator initialisé")

## 4. Fonctions de Génération (Simulées)

Pour la démonstration, nous utilisons des fonctions simulées. En production, remplacez par les vrais appels aux modèles.

In [ ]:
# =============================================================================
# 4. FONCTIONS DE GÉNÉRATION (SIMULÉES)
# =============================================================================

def generate_prompt_variations(base_prompt: str, count: int = 3) -> Dict:
    """Génère des variations d'un prompt (simulé)."""
    time.sleep(0.5)  # Simule un appel API
    
    variations = [
        f"{base_prompt}, photorealistic, 8k",
        f"{base_prompt}, digital art, vibrant colors",
        f"{base_prompt}, oil painting, classic style"
    ][:count]
    
    return {"prompts": variations}

def generate_image_placeholder(prompt: str, model: str = "default", **kwargs) -> Dict:
    """Simule une génération d'image."""
    time.sleep(1 + np.random.random())  # Simule la génération
    
    # Créer une image placeholder colorée
    color = np.random.randint(50, 200, 3)
    img = Image.new('RGB', (512, 512), tuple(color))
    
    return {
        "image": img,
        "model": model,
        "prompt": prompt,
        "seed": kwargs.get("seed", np.random.randint(0, 1000000))
    }

def upscale_image(image: Image.Image, scale: int = 2) -> Dict:
    """Simule un upscaling."""
    time.sleep(0.5)
    
    new_size = (image.width * scale, image.height * scale)
    upscaled = image.resize(new_size, Image.Resampling.LANCZOS)
    
    return {"image": upscaled, "scale": scale}

def apply_style_transfer(image: Image.Image, style: str) -> Dict:
    """Simule un transfert de style."""
    time.sleep(0.8)
    
    # Simuler un effet de style (juste modifier les couleurs)
    arr = np.array(image).astype(float)
    
    if style == "warm":
        arr[:,:,0] = np.clip(arr[:,:,0] * 1.2, 0, 255)  # Plus de rouge
    elif style == "cool":
        arr[:,:,2] = np.clip(arr[:,:,2] * 1.2, 0, 255)  # Plus de bleu
    elif style == "vintage":
        arr = np.clip(arr * 0.9 + 20, 0, 255)  # Délavé
    
    styled = Image.fromarray(arr.astype(np.uint8))
    return {"image": styled, "style": style}

def evaluate_quality(image: Image.Image) -> Dict:
    """Simule une évaluation de qualité."""
    time.sleep(0.3)
    
    # Score aléatoire entre 0.5 et 1.0
    score = 0.5 + np.random.random() * 0.5
    
    return {
        "quality_score": score,
        "passed": score > 0.7,
        "feedback": "Good quality" if score > 0.7 else "Needs improvement"
    }

print("✅ Fonctions de génération définies")

## 5. Pipeline Séquentiel: Text → Image → Style → Upscale

In [ ]:
# =============================================================================
# 5. PIPELINE SÉQUENTIEL
# =============================================================================

print("\n📋 Pipeline Séquentiel: Text → Image → Style → Upscale")

# Définir les étapes
sequential_steps = [
    WorkflowStep(
        name="generate_image",
        func=generate_image_placeholder,
        inputs={"prompt": "A serene mountain landscape at sunset", "model": "sd35"}
    ),
    WorkflowStep(
        name="apply_style",
        func=lambda previous_output, **kw: apply_style_transfer(
            previous_output["image"], style="warm"
        ),
        inputs={}
    ),
    WorkflowStep(
        name="upscale",
        func=lambda previous_output, **kw: upscale_image(
            previous_output["image"], scale=2
        ),
        inputs={}
    ),
    WorkflowStep(
        name="evaluate",
        func=lambda previous_output, **kw: evaluate_quality(previous_output["image"]),
        inputs={}
    )
]

# Exécuter
results = orchestrator.run_sequential(sequential_steps)

# Afficher les résultats
print("\n📊 Résumé:")
summary = orchestrator.get_summary()
for key, value in summary.items():
    print(f"   {key}: {value}")

# Visualiser si succès
if "upscale" in results and results["upscale"].status == TaskStatus.COMPLETED:
    final_image = results["upscale"].output["image"]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    if "generate_image" in results:
        axes[0].imshow(results["generate_image"].output["image"])
        axes[0].set_title("1. Generated")
        axes[0].axis('off')
    
    if "apply_style" in results:
        axes[1].imshow(results["apply_style"].output["image"])
        axes[1].set_title("2. Styled")
        axes[1].axis('off')
    
    axes[2].imshow(final_image)
    axes[2].set_title(f"3. Upscaled ({final_image.size})")
    axes[2].axis('off')
    
    plt.suptitle("Pipeline Séquentiel", fontsize=14)
    plt.tight_layout()
    plt.show()

## 6. Pipeline Parallèle: Génération Multi-Modèles

In [ ]:
# =============================================================================
# 6. PIPELINE PARALLÈLE - MULTI-MODÈLES
# =============================================================================

print("\n⚡ Pipeline Parallèle: Comparaison Multi-Modèles")

shared_prompt = "A futuristic city with flying cars and neon lights"

parallel_steps = [
    WorkflowStep(
        name="qwen_generation",
        func=generate_image_placeholder,
        inputs={"prompt": shared_prompt, "model": "qwen", "seed": 42}
    ),
    WorkflowStep(
        name="flux_generation",
        func=generate_image_placeholder,
        inputs={"prompt": shared_prompt, "model": "flux", "seed": 42}
    ),
    WorkflowStep(
        name="sd35_generation",
        func=generate_image_placeholder,
        inputs={"prompt": shared_prompt, "model": "sd35", "seed": 42}
    ),
]

# Exécuter en parallèle
parallel_results = orchestrator.run_parallel(parallel_steps)

# Affichage
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, (name, result) in enumerate(parallel_results.items()):
    if result.status == TaskStatus.COMPLETED:
        axes[i].imshow(result.output["image"])
        model = result.output["model"]
        axes[i].set_title(f"{model.upper()}\n({result.duration:.2f}s)")
    else:
        axes[i].text(0.5, 0.5, "Failed", ha='center', va='center')
        axes[i].set_title(name)
    axes[i].axis('off')

plt.suptitle(f"Comparaison Multi-Modèles\n'{shared_prompt[:50]}...'", fontsize=12)
plt.tight_layout()
plt.show()

print(f"\n⏱️ Temps total: {orchestrator.get_summary()['total_time']:.2f}s")

## 7. Pipeline Conditionnel: Qualité-Based Routing

In [ ]:
# =============================================================================
# 7. PIPELINE CONDITIONNEL
# =============================================================================

class ConditionalPipeline:
    """
    Pipeline avec branchement conditionnel basé sur la qualité.
    Si qualité < seuil: régénère avec paramètres améliorés.
    """
    
    def __init__(self, quality_threshold: float = 0.7):
        self.quality_threshold = quality_threshold
        self.history = []
    
    def run(self, prompt: str, max_attempts: int = 3) -> Dict:
        """Exécute avec amélioration itérative."""
        print(f"\n🔄 Pipeline Conditionnel (seuil: {self.quality_threshold})")
        print("=" * 40)
        
        best_result = None
        best_score = 0
        
        for attempt in range(max_attempts):
            print(f"\n[Attempt {attempt + 1}/{max_attempts}]")
            
            # Générer
            gen_result = generate_image_placeholder(
                prompt=prompt,
                model="sd35",
                steps=20 + attempt * 10  # Plus de steps à chaque tentative
            )
            print(f"   🖼️ Image générée")
            
            # Évaluer
            eval_result = evaluate_quality(gen_result["image"])
            score = eval_result["quality_score"]
            print(f"   📊 Score: {score:.2f}")
            
            self.history.append({
                "attempt": attempt + 1,
                "score": score,
                "passed": eval_result["passed"]
            })
            
            # Garder le meilleur
            if score > best_score:
                best_score = score
                best_result = gen_result
            
            # Condition d'arrêt
            if score >= self.quality_threshold:
                print(f"   ✅ Qualité suffisante atteinte!")
                break
            else:
                print(f"   ⚠️ Qualité insuffisante, retry...")
        
        return {
            "result": best_result,
            "final_score": best_score,
            "attempts": len(self.history),
            "history": self.history
        }

# Test
conditional = ConditionalPipeline(quality_threshold=0.75)
cond_result = conditional.run("A magical forest with glowing mushrooms")

print(f"\n📋 Résultat Final:")
print(f"   Tentatives: {cond_result['attempts']}")
print(f"   Score final: {cond_result['final_score']:.2f}")

# Visualiser l'historique
if cond_result["history"]:
    attempts = [h["attempt"] for h in cond_result["history"]]
    scores = [h["score"] for h in cond_result["history"]]
    
    plt.figure(figsize=(8, 4))
    plt.bar(attempts, scores, color=['green' if s >= 0.75 else 'orange' for s in scores])
    plt.axhline(y=0.75, color='red', linestyle='--', label='Seuil')
    plt.xlabel('Tentative')
    plt.ylabel('Score Qualité')
    plt.title('Pipeline Conditionnel - Évolution de la Qualité')
    plt.legend()
    plt.show()

## 8. Pipeline Avancé: Génération Multi-Variations

In [ ]:
# =============================================================================
# 8. PIPELINE MULTI-VARIATIONS
# =============================================================================

class MultiVariationPipeline:
    """Pipeline pour générer et comparer plusieurs variations."""
    
    def __init__(self):
        self.orchestrator = WorkflowOrchestrator(max_workers=4)
    
    def generate_variations(self, base_prompt: str, 
                           styles: List[str] = None,
                           models: List[str] = None) -> Dict:
        """Génère des variations par style et/ou modèle."""
        
        styles = styles or ["photorealistic", "digital art", "oil painting"]
        models = models or ["sd35"]
        
        print(f"\n🎨 Génération Multi-Variations")
        print(f"   Base: '{base_prompt[:40]}...'")
        print(f"   Styles: {styles}")
        print(f"   Models: {models}")
        
        # Créer les étapes
        steps = []
        for model in models:
            for style in styles:
                styled_prompt = f"{base_prompt}, {style}"
                steps.append(WorkflowStep(
                    name=f"{model}_{style.replace(' ', '_')}",
                    func=generate_image_placeholder,
                    inputs={"prompt": styled_prompt, "model": model}
                ))
        
        # Exécuter en parallèle
        results = self.orchestrator.run_parallel(steps)
        
        return {
            "variations": results,
            "summary": self.orchestrator.get_summary()
        }
    
    def select_best(self, results: Dict) -> Tuple[str, Any]:
        """Sélectionne la meilleure variation basée sur l'évaluation."""
        best_name = None
        best_score = 0
        
        for name, result in results["variations"].items():
            if result.status == TaskStatus.COMPLETED:
                eval_result = evaluate_quality(result.output["image"])
                if eval_result["quality_score"] > best_score:
                    best_score = eval_result["quality_score"]
                    best_name = name
        
        return best_name, best_score

# Test
multi_pipeline = MultiVariationPipeline()
variations_result = multi_pipeline.generate_variations(
    "A cozy cabin in the snowy mountains",
    styles=["photorealistic", "watercolor", "anime"],
    models=["sd35"]
)

# Affichage grille
completed = {k: v for k, v in variations_result["variations"].items() 
             if v.status == TaskStatus.COMPLETED}

if completed:
    n = len(completed)
    cols = min(3, n)
    rows = (n + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))
    axes = np.array(axes).flatten() if n > 1 else [axes]
    
    for i, (name, result) in enumerate(completed.items()):
        axes[i].imshow(result.output["image"])
        axes[i].set_title(name.replace('_', ' '), fontsize=10)
        axes[i].axis('off')
    
    for i in range(len(completed), len(axes)):
        axes[i].axis('off')
    
    plt.suptitle("Multi-Variations", fontsize=14)
    plt.tight_layout()
    plt.show()

# Sélectionner la meilleure
best_name, best_score = multi_pipeline.select_best(variations_result)
print(f"\n🏆 Meilleure variation: {best_name} (score: {best_score:.2f})")

## 9. Exercices Pratiques

### Exercice 1: Pipeline Personnalisé
Créez un pipeline qui génère une image, applique 3 styles différents, et sélectionne le meilleur.

### Exercice 2: Retry Intelligent
Modifiez `ConditionalPipeline` pour utiliser des paramètres différents à chaque retry.

### Exercice 3: Métriques
Ajoutez des métriques de performance (temps par modèle, taux de succès).

In [ ]:
# =============================================================================
# 9. ESPACE D'EXERCICES
# =============================================================================

# Exercice 1: Décommentez et complétez

# custom_steps = [
#     WorkflowStep(name="generate", func=generate_image_placeholder, 
#                  inputs={"prompt": "votre prompt"}),
#     # Ajoutez vos étapes...
# ]
# 
# custom_orchestrator = WorkflowOrchestrator()
# custom_results = custom_orchestrator.run_sequential(custom_steps)

print("📝 Espace d'exercices - Décommentez pour commencer")

## 10. Récapitulatif

### Patterns d'Orchestration

| Pattern | Cas d'Usage | Avantages |
|---------|-------------|----------|
| **Séquentiel** | Chaînage dépendant | Simple, prévisible |
| **Parallèle** | Comparaison modèles | Rapide, efficace |
| **Conditionnel** | Qualité garantie | Adaptif, robuste |
| **Multi-Variations** | Exploration | Diversité, choix |

### Bonnes Pratiques

1. **Retry avec backoff** pour la résilience
2. **Caching** pour les résultats intermédiaires
3. **Timeouts** pour éviter les blocages
4. **Logging** pour le debugging
5. **Évaluation automatique** pour la sélection

In [ ]:
# =============================================================================
# FIN DU NOTEBOOK
# =============================================================================

print("\n" + "="*60)
print("   ✅ Notebook Workflow Orchestration Complété")
print("="*60)
print(f"\n📅 Terminé: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n📚 Concepts couverts:")
print("   • WorkflowOrchestrator pour pipelines")
print("   • Exécution séquentielle et parallèle")
print("   • Pipelines conditionnels")
print("   • Multi-variations et sélection")
print("   • Retry et gestion d'erreurs")
print("\n➡️  Prochain notebook: 03-3-Performance-Optimization.ipynb")